In [2]:
# Mappings for the encoded values
# So Aiza what we are doing over here is that from our data set ,this is a dictionary basically ( key value pair ), we are putting numeric 
#data so that we can use it.
pclass_mapping = {"First": 1, "Second": 2, "Third": 3}
gender_mapping = {"Male": 1, "Female": 2}
#we observed the data noone had more than 3 siblings 
sibling_mapping = {"Zero": 0, "One": 1, "Two": 2, "Three": 3}
#these are the areas they came from i guess.
embarked_mapping = {"Southampton": 1, "Cherbourg": 2, "Queenstown": 3}
#this is the output column
survived_mapping = {"Yes": 1, "No": 0}

# Sample data (hardcoded)
data = [
    #these are the rows in the form of key value pair 
    {"PClass": "First", "Gender": "Male", "Sibling": "Zero", "Embarked": "Southampton", "Survived": "Yes"},
    {"PClass": "Second", "Gender": "Female", "Sibling": "One", "Embarked": "Cherbourg", "Survived": "No"},
    {"PClass": "Third", "Gender": "Male", "Sibling": "Two", "Embarked": "Queenstown", "Survived": "Yes"},
    # Add more data as needed...
]

# Encode the data manually
def encode_data(data):
    #here we are encoding our data by this i mean that we would get a list or dictionary and we would pick each value of the key
    # and replace it with the value of that key ( that we have as a value here), pclass_mapping is a dictionary.we would do this with all
    # the rows okay ? 
    for entry in data:
        entry["encoded_pclass"] = pclass_mapping[entry["PClass"]]
        entry["encoded_gender"] = gender_mapping[entry["Gender"]]
        entry["encoded_sibling"] = sibling_mapping[entry["Sibling"]]
        entry["encoded_embarked"] = embarked_mapping[entry["Embarked"]]
        entry["encoded_survived"] = survived_mapping[entry["Survived"]]

encode_data(data)

# Feature Scaling: Min-Max Normalization
def scale_features(data, feature_keys):
    #this is a dictionary , whats feature keys ? I think its value of the features for that specific row? whats (inf)?
    min_max = {key: (float('inf'), float('-inf')) for key in feature_keys}
    #for every row in dataset , and then every key in row ?
    for entry in data:
        for key in feature_keys:
            value = entry[key]
            min_max[key] = (min(min_max[key][0], value), max(min_max[key][1], value))
    for entry in data:
        for key in feature_keys:
            min_val, max_val = min_max[key]
            if max_val != min_val:
                entry[key] = (entry[key] - min_val) / (max_val - min_val)
            else:
                entry[key] = 0.0

feature_keys = ["encoded_pclass", "encoded_gender", "encoded_sibling", "encoded_embarked"]
scale_features(data, feature_keys)

# Logistic Regression implementation
class LogisticRegressionManual:
    #okay so made this a class and it has methods 
    def __init__(self, learning_rate=0.01, iterations=2000):  # Increased iterations
        self.learning_rate = learning_rate
        self.iterations = iterations

    def sigmoid(self, z):
        return 1 / (1 + self.exp(-z))

    def exp(self, value):
        return sum([(value ** n) / self.factorial(n) for n in range(20)])

    def factorial(self, n):
        return 1 if n == 0 else n * self.factorial(n-1)

    def train(self, X, y):
        self.m = len(X)
        self.n = len(X[0])
        self.weights = [0] * self.n
        self.bias = 0
        for _ in range(self.iterations):
            linear_model = [self.dot_product(self.weights, X[i]) + self.bias for i in range(self.m)]
            y_predicted = [self.sigmoid(linear_model[i]) for i in range(self.m)]

            dw = [(1 / self.m) * sum([(y_predicted[i] - y[i]) * X[i][j] for i in range(self.m)]) for j in range(self.n)]
            db = (1 / self.m) * sum([(y_predicted[i] - y[i]) for i in range(self.m)])

            self.weights = [self.weights[j] - self.learning_rate * dw[j] for j in range(self.n)]
            self.bias -= self.learning_rate * db

    def dot_product(self, vector1, vector2):
        return sum([vector1[i] * vector2[i] for i in range(len(vector1))])

    def predict(self, X):
        linear_model = [self.dot_product(self.weights, X[i]) + self.bias for i in range(len(X))]
        y_predicted = [self.sigmoid(linear_model[i]) for i in range(len(X))]
        return [1 if i > 0.5 else 0 for i in y_predicted]

# K-Nearest Neighbors implementation
class KNNManual:
    def __init__(self, k=3):
        self.k = k

    def train(self, X, y):
        self.X_train = X
        self.y_train = y

    def euclidean_distance(self, x1, x2):
        return sum([(x1[i] - x2[i]) ** 2 for i in range(len(x1))]) ** 0.5

    def predict(self, X):
        predictions = []
        for x in X:
            distances = [(self.euclidean_distance(x, self.X_train[i]), self.y_train[i]) for i in range(len(self.X_train))]
            distances.sort(key=lambda x: x[0])
            k_nearest_labels = [distances[i][1] for i in range(self.k)]
            most_common = max(set(k_nearest_labels), key=k_nearest_labels.count)
            predictions.append(most_common)
        return predictions

# Prepare the data
X = [[entry["encoded_pclass"], entry["encoded_gender"], entry["encoded_sibling"], entry["encoded_embarked"]] for entry in data]
y = [entry["encoded_survived"] for entry in data]

# Train and test split (for simplicity, we use all data for training and testing)
X_train = X
y_train = y
X_test = X
y_test = y

# Train Logistic Regression model
log_reg_model = LogisticRegressionManual()
log_reg_model.train(X_train, y_train)
log_reg_predictions = log_reg_model.predict(X_test)

# Calculate Logistic Regression accuracy
log_reg_accuracy = sum([1 for i in range(len(y_test)) if y_test[i] == log_reg_predictions[i]]) / len(y_test)
print("\nLogistic Regression Accuracy Score:", round(log_reg_accuracy, 2))

# Train KNN model
knn_model = KNNManual(k=3)
knn_model.train(X_train, y_train)
knn_predictions = knn_model.predict(X_test)

# Calculate KNN accuracy
knn_accuracy = sum([1 for i in range(len(y_test)) if y_test[i] == knn_predictions[i]]) / len(y_test)
print("\nKNN Accuracy Score:", round(knn_accuracy, 2))

# Predict for a new user input
user_input = {"PClass": "First", "Gender": "Male", "Sibling": "Zero", "Embarked": "Southampton"}
user_input_encoded = [
    pclass_mapping[user_input["PClass"]],
    gender_mapping[user_input["Gender"]],
    sibling_mapping[user_input["Sibling"]],
    embarked_mapping[user_input["Embarked"]]
]

# Feature scaling for user input
user_input_encoded = [
    (user_input_encoded[i] - min([x[i] for x in X_train])) / (max([x[i] for x in X_train]) - min([x[i] for x in X_train]))
    for i in range(len(user_input_encoded))
]

log_reg_result = log_reg_model.predict([user_input_encoded])[0]
knn_result = knn_model.predict([user_input_encoded])[0]

print(f"\nLogistic Regression Prediction: {'SURVIVED' if log_reg_result == 1 else 'NOT SURVIVED'}")
print(f"KNN Prediction: {'SURVIVED' if knn_result == 1 else 'NOT SURVIVED'}")



Logistic Regression Accuracy Score: 1.0

KNN Accuracy Score: 0.67

Logistic Regression Prediction: NOT SURVIVED
KNN Prediction: SURVIVED


In [6]:
# Mappings for the encoded values
pclass_mapping = {"First": 1, "Second": 2, "Third": 3}
gender_mapping = {"Male": 1, "Female": 2}
sibling_mapping = {"Zero": 0, "One": 1, "Two": 2, "Three": 3}
embarked_mapping = {"Southampton": 1, "Cherbourg": 2, "Queenstown": 3}
survived_mapping = {"Yes": 1, "No": 0}

# Sample data (hardcoded)
data = [
    {"PClass": "First", "Gender": "Male", "Sibling": "Zero", "Embarked": "Southampton", "Survived": "Yes"},
    {"PClass": "Second", "Gender": "Female", "Sibling": "One", "Embarked": "Cherbourg", "Survived": "No"},
    {"PClass": "Third", "Gender": "Male", "Sibling": "Two", "Embarked": "Queenstown", "Survived": "Yes"},
    {"PClass": "First", "Gender": "Female", "Sibling": "One", "Embarked": "Southampton", "Survived": "Yes"},
    {"PClass": "First", "Gender": "Female", "Sibling": "Two", "Embarked": "Cherbourg", "Survived": "No"},
    {"PClass": "Second", "Gender": "Male", "Sibling": "Two", "Embarked": "Southampton", "Survived": "Yes"},
    {"PClass": "First", "Gender": "Female", "Sibling": "Zero", "Embarked": "Cherbourg", "Survived": "No"},
    {"PClass": "Third", "Gender": "Male", "Sibling": "Two", "Embarked": "Southampton", "Survived": "Yes"},
    {"PClass": "Second", "Gender": "Female", "Sibling": "Zero", "Embarked": "Queenstown", "Survived": "No"},
    {"PClass": "Third", "Gender": "Male", "Sibling": "Two", "Embarked": "Cherbourg", "Survived": "Yes"},
    {"PClass": "Second", "Gender": "Female", "Sibling": "One", "Embarked": "Southampton", "Survived": "No"},
    {"PClass": "First", "Gender": "Male", "Sibling": "Zero", "Embarked": "Queenstown", "Survived": "Yes"},
    
    # Add more data as needed...
]

# Encode the data manually
def encode_data(data):
    for entry in data:
        entry["encoded_pclass"] = pclass_mapping[entry["PClass"]]
        entry["encoded_gender"] = gender_mapping[entry["Gender"]]
        entry["encoded_sibling"] = sibling_mapping[entry["Sibling"]]
        entry["encoded_embarked"] = embarked_mapping[entry["Embarked"]]
        entry["encoded_survived"] = survived_mapping[entry["Survived"]]

encode_data(data)

# Feature Scaling: Min-Max Normalization
def scale_features(data, feature_keys):
    min_max = {key: (float('inf'), float('-inf')) for key in feature_keys}
    for entry in data:
        for key in feature_keys:
            value = entry[key]
            min_max[key] = (min(min_max[key][0], value), max(min_max[key][1], value))
    for entry in data:
        for key in feature_keys:
            min_val, max_val = min_max[key]
            if max_val != min_val:
                entry[key] = (entry[key] - min_val) / (max_val - min_val)
            else:
                entry[key] = 0.0

feature_keys = ["encoded_pclass", "encoded_gender", "encoded_sibling", "encoded_embarked"]
scale_features(data, feature_keys)

# Logistic Regression implementation
class LogisticRegressionManual:
    def __init__(self, learning_rate=0.01, iterations=2000):  # Increased iterations
        self.learning_rate = learning_rate
        self.iterations = iterations

    def sigmoid(self, z):
        return 1 / (1 + self.exp(-z))

    def exp(self, value):
        return sum([(value ** n) / self.factorial(n) for n in range(20)])

    def factorial(self, n):
        return 1 if n == 0 else n * self.factorial(n-1)

    def train(self, X, y):
        self.m = len(X)
        self.n = len(X[0])
        self.weights = [0] * self.n
        self.bias = 0
        for _ in range(self.iterations):
            linear_model = [self.dot_product(self.weights, X[i]) + self.bias for i in range(self.m)]
            y_predicted = [self.sigmoid(linear_model[i]) for i in range(self.m)]

            dw = [(1 / self.m) * sum([(y_predicted[i] - y[i]) * X[i][j] for i in range(self.m)]) for j in range(self.n)]
            db = (1 / self.m) * sum([(y_predicted[i] - y[i]) for i in range(self.m)])

            self.weights = [self.weights[j] - self.learning_rate * dw[j] for j in range(self.n)]
            self.bias -= self.learning_rate * db

    def dot_product(self, vector1, vector2):
        return sum([vector1[i] * vector2[i] for i in range(len(vector1))])

    def predict(self, X):
        linear_model = [self.dot_product(self.weights, X[i]) + self.bias for i in range(len(X))]
        y_predicted = [self.sigmoid(linear_model[i]) for i in range(len(X))]
        return [1 if i > 0.5 else 0 for i in y_predicted]

# K-Nearest Neighbors implementation
class KNNManual:
    def __init__(self, k=3):
        self.k = k

    def train(self, X, y):
        self.X_train = X
        self.y_train = y

    def euclidean_distance(self, x1, x2):
        return sum([(x1[i] - x2[i]) ** 2 for i in range(len(x1))]) ** 0.5

    def predict(self, X):
        predictions = []
        for x in X:
            distances = [(self.euclidean_distance(x, self.X_train[i]), self.y_train[i]) for i in range(len(self.X_train))]
            distances.sort(key=lambda x: x[0])
            k_nearest_labels = [distances[i][1] for i in range(self.k)]
            most_common = max(set(k_nearest_labels), key=k_nearest_labels.count)
            predictions.append(most_common)
        return predictions

# Prepare the data
X = [[entry["encoded_pclass"], entry["encoded_gender"], entry["encoded_sibling"], entry["encoded_embarked"]] for entry in data]
y = [entry["encoded_survived"] for entry in data]

# Train and test split (for simplicity, we use all data for training and testing)
X_train = X
y_train = y
X_test = X
y_test = y

# Train Logistic Regression model
log_reg_model = LogisticRegressionManual()
log_reg_model.train(X_train, y_train)
log_reg_predictions = log_reg_model.predict(X_test)

# Train KNN model
knn_model = KNNManual(k=3)
knn_model.train(X_train, y_train)
knn_predictions = knn_model.predict(X_test)

# Take input from the user
user_input = {}
user_input["PClass"] = input("Enter PClass (First, Second, Third): ")
user_input["Gender"] = input("Enter Gender (Male, Female): ")
user_input["Sibling"] = input("Enter Sibling (Zero, One, Two, Three): ")
user_input["Embarked"] = input("Enter Embarked (Southampton, Cherbourg, Queenstown): ")

user_input_encoded = [
    pclass_mapping[user_input["PClass"]],
    gender_mapping[user_input["Gender"]],
    sibling_mapping[user_input["Sibling"]],
    embarked_mapping[user_input["Embarked"]]
]

# Feature scaling for user input
user_input_encoded = [
    (user_input_encoded[i] - min([x[i] for x in X_train])) / (max([x[i] for x in X_train]) - min([x[i] for x in X_train]))
    for i in range(len(user_input_encoded))
]

# Calculate accuracy before predicting
log_reg_accuracy = sum([1 for i in range(len(y_test)) if y_test[i] == log_reg_predictions[i]]) / len(y_test)
knn_accuracy = sum([1 for i in range(len(y_test)) if y_test[i] == knn_predictions[i]]) / len(y_test)

print("\nLogistic Regression Accuracy Score:", round(log_reg_accuracy, 2))
print("KNN Accuracy Score:", round(knn_accuracy, 2))

# Predict using Logistic Regression
log_reg_result = log_reg_model.predict([user_input_encoded])[0]

# Predict using KNN
knn_result = knn_model.predict([user_input_encoded])[0]

# Display predictions
print(f"\nLogistic Regression Prediction: {'SURVIVED' if log_reg_result == 1 else 'NOT SURVIVED'}")
print(f"KNN Prediction: {'SURVIVED' if knn_result == 1 else 'NOT SURVIVED'}")

Enter PClass (First, Second, Third): First
Enter Gender (Male, Female): Female
Enter Sibling (Zero, One, Two, Three): Two
Enter Embarked (Southampton, Cherbourg, Queenstown): Southampton

Logistic Regression Accuracy Score: 0.92
KNN Accuracy Score: 0.92

Logistic Regression Prediction: NOT SURVIVED
KNN Prediction: NOT SURVIVED
